In [0]:
"""
    Author: Moustafa Alzantot (malzantot@ucla.edu)
    All rights reserved.
"""
import os
import data_utils
import pickle

from keras.preprocessing.sequence import pad_sequences

import numpy as np
import tensorflow as tf
#tf.disable_v2_behavior()
import pickle

import models
from importlib import reload
reload(models)

tf.compat.v1.disable_eager_execution()


Using TensorFlow backend.


In [0]:
from google.colab import files
upload = files.upload()

Saving data_utils.py to data_utils.py


In [0]:

IMDB_PATH = 'aclImdb'
MAX_VOCAB_SIZE = 50000

if __name__ == '__main__':
    with open(('/content/var/dataset_%d.pkl' %MAX_VOCAB_SIZE), 'rb') as f:
        dataset = pickle.load(f)
    # TODO(malzantot): should we keep using the normal glove embeddings here ?
    embedding_matrix = np.load(('/content/var/embeddings_glove_%d.npy' %(MAX_VOCAB_SIZE)))
    max_len = 250
    train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
    train_y = np.array(dataset.train_y)
    test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
    test_y = np.array(dataset.test_y)
    sess = tf.compat.v1.Session()
    batch_size = 64
    lstm_size = 128
    num_epochs = 20
    with tf.compat.v1.variable_scope('imdb', reuse=False):
        model = models.SentimentModel(batch_size=batch_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           keep_probs=0.8,
                           embeddings_dim=300, vocab_size=embedding_matrix.shape[1],
			   is_train = True)
    with tf.compat.v1.variable_scope('imdb', reuse=True):
        test_model = models.SentimentModel(batch_size=batch_size,
                                lstm_size = lstm_size,
                                max_len = max_len,keep_probs=0.8,
                                embeddings_dim=300, vocab_size=embedding_matrix.shape[1],is_train= False)
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(tf.compat.v1.assign(model.embedding_w, embedding_matrix.T))
    print('Training..')
    for i in range(num_epochs):
        epoch_loss, epoch_accuracy = model.train_for_epoch(sess, train_x, train_y)
        print(i, ' ', epoch_loss, ' ', epoch_accuracy)
        print('Train accuracy = ', test_model.evaluate_accuracy(sess, train_x, train_y))
        print('Test accuracy = ', test_model.evaluate_accuracy(sess, test_x, test_y))
    if not os.path.exists('models'):
        os.mkdir('models')
    saver = tf.train.Saver()
    saver.save(sess, 'models/imdb_model')
    model.save('imdb_model.h5')
    print('All done')
    
    #with open('model_pickle','wb') as f:
        #pickle.dump(model,f)


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Training..
0   0.6549518115245379   0.6188301282051282
Train accuracy =  0.7907051282051282
Test accuracy =  0.7776041666666667
1   0.43245482509716965   0.809775641025641
Train accura

AttributeError: ignored